In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd 

from avoca.bindings.gcwerks import read_gcwerks, export

from pathlib import Path
from datetime import datetime


In [ ]:


workdir = Path('./KSJ_flask')
workdir.mkdir(exist_ok=True)
temp_file = workdir / 'sevoflurane.dat'
export(
    workdir=workdir,
    gcdir="/agage/empa-medusa", 
    compounds=['sevoflurane'], 
    out_file=temp_file, 
    date_start=datetime(2010, 1, 1),
    variables=['C', 'RL.report' ]
    
)
df_gcwerks = read_gcwerks(temp_file, runtypes=['tank'])
df_gcwerks

In [ ]:
mini_archive_file = "/agage/empa-medusa/logs/hardware.log/MiniArchive2013"
lines = []

columns = [
    'measurement_time',
    'sample_id',
    'tank_id',
    'tank_material',
    'pressure',
    'lon',
    'lat',
    'sampling_time',
    'station', 
    'filling_method', 
    'sampling_volume',
    'operator',
]
with open(mini_archive_file, 'r') as f:
    for i, line in enumerate(f):
        if line.startswith('#'):
            continue
        if line.strip() == '':
            continue
        values = line.strip().split()
        if len(values) < len(columns):
            print(f"Skipping line {line} with insufficient values: {line.strip()}")
            continue
        lines.append(values[:len(columns)])

lines
df = pd.DataFrame(lines, columns=columns)
df

In [ ]:
df['tank_material'].unique()
df['sampling_time'] = pd.to_datetime(df['sampling_time'], format='%Y%m%d')
df['measurement_time'] = pd.to_datetime(df['measurement_time'], format='%y%m%d.%H%M')
df_ksg = df.loc[df['station'] == 'KSG']
df_ksg

In [ ]:
df_gcwerks_ksg = df_gcwerks.loc[df_gcwerks[('-', 'sample')].str.startswith('KSG')].copy()

# access the string from 3:9 of the column 
df_gcwerks_ksg['sampling_time'] = pd.to_datetime(df_gcwerks_ksg[('-', 'sample')].str[3:9], format='%y%m%d')
df_gcwerks_ksg

In [ ]:
df_gcwerks_ksg

In [ ]:
plt.figure(figsize=(10, 6))
df_gcwerks_ksg.plot(x='sampling_time', y=('sevoflurane', 'C'), kind='scatter', title='KSG Sevoflurane Concentration')
plt.savefig(workdir / 'KSG_sevoflurane_concentration.png')
plt.show()